Ingestão e Exploração dos Dados

Configuração Inicial

In [1]:
# Célula 1: Imports e Configurações
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

# Configurações visuais
sns.set_theme(style="whitegrid", palette="husl")
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['font.size'] = 11

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


Carregamento dos Dados

In [3]:
# Célula 2: Carregamento dos JSONs
df_accounts = pd.read_json('../data/raw/accounts_anonymized.json')
df_cases = pd.read_json('../data/raw/support_cases_anonymized.json')

# Converter colunas de data imediatamente (JSON carrega datas como strings)
# Isso facilita o trabalho com SQL mais tarde
df_accounts['account_created_date'] = pd.to_datetime(df_accounts['account_created_date'])
df_cases['case_created_date'] = pd.to_datetime(df_cases['case_created_date'])
df_cases['case_closed_date'] = pd.to_datetime(df_cases['case_closed_date'])

print(f"📊 Accounts carregados: {len(df_accounts)} registros")
print(f"📊 Cases carregados: {len(df_cases)} registros")

📊 Accounts carregados: 1415 registros
📊 Cases carregados: 10000 registros


Análise Exploratória Detalhada

In [5]:
# Célula 3: Estrutura e Qualidade - Accounts
print("=" * 80)
print("ANÁLISE: ACCOUNTS")
print("=" * 80)

print("\n📋 Informações Gerais:")
print(df_accounts.info())

print("\n📊 Resumo Categórico (Top valores):")
cols_cat = ['account_country', 'account_industry']
display(df_accounts[cols_cat].describe())

print("\n📅 Resumo Temporal:")
print(f"Primeira conta criada em: {df_accounts['account_created_date'].min()}")
print(f"Última conta criada em:   {df_accounts['account_created_date'].max()}")
print(f"Período total de dados:   {df_accounts['account_created_date'].max() - df_accounts['account_created_date'].min()}")

print("\n🔍 Primeiras 5 linhas:")
display(df_accounts.head())

print("\n⚠️ Valores Nulos:")
print(df_accounts.isnull().sum())

print("\n🔑 Colunas disponíveis:")
print(df_accounts.columns.tolist())

ANÁLISE: ACCOUNTS

📋 Informações Gerais:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415 entries, 0 to 1414
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   account_sfid          1415 non-null   object        
 1   account_name          1415 non-null   object        
 2   account_created_date  1415 non-null   datetime64[ns]
 3   account_country       1408 non-null   object        
 4   account_industry      1402 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 55.4+ KB
None

📊 Resumo Categórico (Top valores):


,account_country,account_industry
count,1408,1402
unique,72,22
top,United States,Pharmaceuticals
freq,553,421



📅 Resumo Temporal:
Primeira conta criada em: 2007-11-19 21:27:02
Última conta criada em:   2025-01-06 10:46:57
Período total de dados:   6257 days 13:19:55

🔍 Primeiras 5 linhas:


,account_sfid,account_name,account_created_date,account_country,account_industry
0,28cb93e07db33a85d12c75825a11ab6a97c5a0df9750f6...,Customer_b9c711e0,2017-12-18 15:53:15,Pakistan,Printing
1,17779701277914218bcfcd9211c3fb6673758ca2f6e014...,Customer_b3e69ecd,2010-01-11 17:41:46,United Kingdom,Household & Personal Products
2,777c8f6154d9e1c1563ce0a62dbd8392ef3204cec6ecd1...,Customer_3b6e1b44,2020-10-19 17:12:22,China,Packaging and Containers
3,e7b6a33c931f01e1e2d42b47318945216da6fef294a738...,Customer_58de6751,2019-03-20 12:56:09,South Korea,Pharmaceuticals
4,fbcec0b147243cfce96df94fe543968c04cfc0a4a29fd7...,Customer_97f3101d,2021-03-24 14:06:24,Poland,Printing



⚠️ Valores Nulos:
account_sfid             0
account_name             0
account_created_date     0
account_country          7
account_industry        13
dtype: int64

🔑 Colunas disponíveis:
['account_sfid', 'account_name', 'account_created_date', 'account_country', 'account_industry']


In [10]:
# Célula 4: Estrutura e Qualidade - Cases
print("=" * 80)
print("ANÁLISE: SUPPORT CASES")
print("=" * 80)

print("\n📋 Informações Gerais:")
print(df_cases.info())

cols_negocio = ['case_status', 'case_priority', 'case_severity', 'case_product']

print("\n📊 Distribuição de Métricas Chave (Top 5):")
for col in cols_negocio:
    print(f"\n--- {col.upper()} ---")
    # Mostra contagem e % relativa lado a lado
    dist = pd.concat([df_cases[col].value_counts(), 
                      df_cases[col].value_counts(normalize=True).mul(100).round(1)], 
                     axis=1, keys=['Qtd', '%'])
    display(dist)

print("\n📅 Resumo Temporal dos Casos:")
print(f"Primeiro caso: {df_cases['case_created_date'].min()}")
print(f"Último caso:   {df_cases['case_created_date'].max()}")

print("\n⚠️ Valores Nulos (Importante para identificar casos abertos):")
print(df_cases.isnull().sum())

ANÁLISE: SUPPORT CASES

📋 Informações Gerais:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   case_sfid          10000 non-null  object        
 1   account_sfid       8407 non-null   object        
 2   case_number        10000 non-null  int64         
 3   case_contact_name  10000 non-null  object        
 4   case_product       10000 non-null  object        
 5   case_status        10000 non-null  object        
 6   case_resolution    10000 non-null  object        
 7   case_details       10000 non-null  object        
 8   case_closure       10000 non-null  object        
 9   case_priority      10000 non-null  object        
 10  case_severity      10000 non-null  object        
 11  case_reason        10000 non-null  object        
 12  case_type          10000 non-null  object        
 13  case_category   

,Qtd,%
case_status,,
Closed,7043,70.4
Duplicate,2015,20.2
New,795,8.0
Waiting On Customer,45,0.4
Working,39,0.4
Sent_To_Dev,35,0.4
Waiting for 3rd Party,10,0.1
Remote Session Scheduled,9,0.1
Assigned,9,0.1



--- CASE_PRIORITY ---


,Qtd,%
case_priority,,
Normal,9811,98.1
Urgent,152,1.5
not_priority,35,0.4
High,2,0.0



--- CASE_SEVERITY ---


,Qtd,%
case_severity,,
Medium,4078,40.8
Low,3159,31.6
Undefined,2659,26.6
High,72,0.7
Urgent,32,0.3



--- CASE_PRODUCT ---


,Qtd,%
case_product,,
GVD,2850,28.5
Other,2573,25.7
Verify,1028,10.3
Not a Support Case,1024,10.2
GVW,825,8.2
...,...,...
F3 Flatbed Scanner,1,0.0
Print Inspection- Named Desktop Perpetual License,1,0.0
Docu-Proof Enterprise Workstation License,1,0.0



📅 Resumo Temporal dos Casos:
Primeiro caso: 2023-11-22 00:00:00
Último caso:   2025-01-09 00:00:00

⚠️ Valores Nulos (Importante para identificar casos abertos):
case_sfid               0
account_sfid         1593
case_number             0
case_contact_name       0
case_product            0
case_status             0
case_resolution         0
case_details            0
case_closure            0
case_priority           0
case_severity           0
case_reason             0
case_type               0
case_category           0
case_created_date       0
case_closed_date      942
dtype: int64


Identificação de Relacionamentos

In [15]:
# Célula 5: Verificação de Integridade (Abordagem Vetorizada)

# 1. Identificar IDs válidos (Conjunto de referência)
valid_account_ids = set(df_accounts['account_sfid'])

# 2. Criar a coluna de status com um valor padrão
df_cases['integrity_status'] = 'Valid Link'

# 3. Marcar os Nulos (Rápido e direto)
df_cases.loc[df_cases['account_sfid'].isnull(), 'integrity_status'] = 'Orphan (Null ID)'

# 4. Marcar os Links Quebrados (IDs que não são nulos, mas não estão na lista de contas)
# O operador ~ significa "NÃO". Ou seja: Onde o ID NÃO está em valid_account_ids
broken_link_mask = (~df_cases['account_sfid'].isin(valid_account_ids)) & (df_cases['account_sfid'].notnull())
df_cases.loc[broken_link_mask, 'integrity_status'] = 'Orphan (Broken Link)'

# === Relatório ===
print("=== Relatório de Integridade ===")
print(df_cases['integrity_status'].value_counts())

# Exibir amostra dos problemas, se houver
orphans = df_cases[df_cases['integrity_status'] != 'Valid Link']
if not orphans.empty:
    print(f"\nAlerta: Encontrados {len(orphans)} registros órfãos.")
    display(orphans[['case_number', 'account_sfid', 'integrity_status']].head())

=== Relatório de Integridade ===
integrity_status
Valid Link          8407
Orphan (Null ID)    1593
Name: count, dtype: int64

Alerta: Encontrados 1593 registros órfãos.


,case_number,account_sfid,integrity_status
4,69891,None,Orphan (Null ID)
10,69892,None,Orphan (Null ID)
17,69902,None,Orphan (Null ID)
25,69911,None,Orphan (Null ID)
26,69822,None,Orphan (Null ID)


In [16]:
# Célula 6: Tratamento dos Órfãos (Data Cleaning)

# Em vez de apagar, vamos preencher os Nulos para evitar erros no SQL depois
df_cases['account_sfid'] = df_cases['account_sfid'].fillna('UNKNOWN_ACCOUNT')

# Opcional: Se quiser ser muito proativo, crie uma conta "fictícia" no df_accounts
# para que o JOIN no SQL não descarte esses dados.
unknown_account = {
    'account_sfid': 'UNKNOWN_ACCOUNT',
    'account_name': 'Unassigned / Data Error',
    'account_industry': 'Unknown',
    'account_country': 'Unknown'
}

# Adiciona essa conta "coringa" ao DataFrame de contas se ela não existir
if 'UNKNOWN_ACCOUNT' not in df_accounts['account_sfid'].values:
    df_accounts = pd.concat([df_accounts, pd.DataFrame([unknown_account])], ignore_index=True)

print("Limpeza realizada: Órfãos mapeados para 'UNKNOWN_ACCOUNT'.")

Limpeza realizada: Órfãos mapeados para 'UNKNOWN_ACCOUNT'.


Processamento com SQL 

Setup do Banco de Dados In-Memory

In [19]:
# Célula 7: Criação do Banco SQLite em Memória
conn = sqlite3.connect(':memory:')

# Carregando dados no SQLite
df_accounts.to_sql('accounts', conn, index=False, if_exists='replace')
df_cases.to_sql('cases', conn, index=False, if_exists='replace')

# Verificando tabelas criadas
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
print("✅ Tabelas criadas no SQLite:")
print(tables)

✅ Tabelas criadas no SQLite:
       name
0  accounts
1     cases


3.2 Queries Analíticas (KPIs de Negócio)

KPI 1: Performance por Indústria

In [22]:
# Célula 8: Análise de Volume e Tempo por Indústria
query_industry = """
SELECT 
    a.account_industry as industry,
    COUNT(c.case_sfid) as total_cases,
    COUNT(DISTINCT c.account_sfid) as unique_accounts,
    -- SQLite usa JULIANDAY para diferença de datas
    ROUND(AVG(JULIANDAY(c.case_closed_date) - JULIANDAY(c.case_created_date)), 2) as avg_resolution_days,
    COUNT(CASE WHEN c.case_priority = 'High' THEN 1 END) as high_priority_cases,
    ROUND(COUNT(CASE WHEN c.case_priority = 'High' THEN 1 END) * 100.0 / COUNT(c.case_sfid), 2) as pct_high_priority
FROM 
    accounts a
JOIN 
    cases c ON a.account_sfid = c.account_sfid
WHERE 
    c.case_sfid IS NOT NULL
GROUP BY 
    a.account_industry
ORDER BY 
    total_cases DESC;
"""

# Usando nossa função auxiliar criada anteriormente (ou pd.read_sql)
df_industry_metrics = pd.read_sql(query_industry, conn)

print("📊 KPI 1: Métricas por Indústria")
display(df_industry_metrics)

📊 KPI 1: Métricas por Indústria


,industry,total_cases,unique_accounts,avg_resolution_days,high_priority_cases,pct_high_priority
0,Pharmaceuticals,2393,421,6.68,0,0.00
1,Information Technology,1849,29,0.90,0,0.00
2,Unknown,1593,1,0.01,0,0.00
3,Printing,1154,265,5.44,0,0.00
4,Packaging and Containers,1072,252,5.73,2,0.19
5,Advertising & Branding Agency,475,66,5.61,0,0.00
6,Household & Personal Products,384,86,6.97,0,0.00
7,Other,308,56,4.24,0,0.00
8,Medical Devices,275,60,4.33,0,0.00
9,Food & Beverage,206,78,3.73,0,0.00
